# Azure AI Content Understanding

This notebook demonstrates how to use the Azure AI Content Understanding service
in Foundry Tools. Content Understanding provides powerful capabilities for analyzing
and extracting information from documents and images.

## Prerequisites

Set the following environment variables:
- `FOUNDRY_CONTENT_UNDERSTANDING_ENDPOINT` - Azure AI Content Understanding service endpoint
- `FOUNDRY_API_KEY` - API key for authentication
- `FOUNDRY_REGION` - Region in which Foundry resources are provisioned

In [ ]:
import json
import os
import time

import requests
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from tqdm.notebook import tqdm

## API Setup

Configure the Azure AI Content Understanding API endpoint and authentication.

In [ ]:
endpoint = os.environ["FOUNDRY_CONTENT_UNDERSTANDING_ENDPOINT"]
api_key = os.environ["FOUNDRY_API_KEY"]
region = os.environ["FOUNDRY_REGION"]

headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/json",
}

console = Console()

print(f"API Endpoint: {endpoint}")
print(f"Region: {region}")
print("API Key: [CONFIGURED]")

# Capability 1: List Content Analyzers

The Content Understanding service provides various prebuilt analyzers for different
document and image analysis tasks. Let's discover what analyzers are available.

In [ ]:
# List all available content analyzers
list_url = f"{endpoint}/contentunderstanding/analyzers?api-version=2025-11-01"

response = requests.get(list_url, headers=headers, timeout=30)
response.raise_for_status()
analyzers_result = response.json()

print("API call successful!")

## Raw API Results - Available Analyzers

In [ ]:
print(json.dumps(analyzers_result, indent=2))

## Human-Friendly Results - Available Analyzers

In [ ]:
console = Console()

# Create a table of available analyzers
table = Table(title="Available Content Analyzers")
table.add_column("Analyzer ID", style="cyan", no_wrap=True)
table.add_column("Description", style="white")

for analyzer in analyzers_result.get("value", []):
    analyzer_id = analyzer.get("analyzerId", "N/A")
    description = analyzer.get("description", "No description available")
    table.add_row(analyzer_id, description)

console.print(table)
console.print(f"\n[bold green]Total Analyzers Available: {len(analyzers_result.get('value', []))}[/bold green]")

# Capability 2: Content Extraction

Extract annotations and highlights from a PDF document using the prebuilt-layout analyzer.
This is useful for extracting marked content, comments, and highlighted sections from documents.

## Prepare PDF Analysis Request

We'll analyze a PDF ebook that contains highlights using the prebuilt-layout analyzer.

In [ ]:
# PDF URL with highlights
pdf_url = "https://www.dropbox.com/scl/fi/405z56tds3tewycemxp23/Azure-AI-Fundamentals-AI-900-Study-Guide-1.pdf?rlkey=i0t4xnrrt392lba4s4fqlt1j7&st=yomn8xte&dl=1"

# Download the PDF file
print(f"Downloading PDF from: {pdf_url}")
pdf_response = requests.get(pdf_url, timeout=60)
pdf_response.raise_for_status()
pdf_data = pdf_response.content
print(f"Downloaded {len(pdf_data)} bytes")

# Endpoint for binary content analysis
analyze_url = f"{endpoint}/contentunderstanding/analyzers/prebuilt-layout:analyzeBinary?api-version=2025-11-01&features=annotations"

# Prepare headers for binary upload
pdf_headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/pdf",
}

print("\nAnalyzer: prebuilt-layout")
print("Features: annotations")

## Make PDF Analysis API Call

Submit the document for analysis. This is an asynchronous operation.

In [ ]:
# Submit the analysis request with binary data
response = requests.post(analyze_url, headers=pdf_headers, data=pdf_data, timeout=30)
response.raise_for_status()

# Get the operation location to poll for results
operation_location = response.headers.get("Operation-Location")

# Poll for results with progress bar
max_attempts = 30
pdf_result = None
final_status = None

for attempt in tqdm(range(max_attempts), desc="Polling for results", unit="attempt"):
    time.sleep(2)
    
    result_response = requests.get(operation_location, headers=headers, timeout=30)
    result_response.raise_for_status()
    pdf_result = result_response.json()
    
    final_status = pdf_result.get("status", "").lower()
    
    if final_status in ["succeeded", "failed"]:
        break

# Print final status after progress bar completes
if final_status == "succeeded":
    print("\n✓ Analysis completed successfully!")
elif final_status == "failed":
    print("\n✗ Analysis failed!")
else:
    print("\n⚠ Timeout waiting for results")

## Raw API Results - PDF Annotations

In [ ]:
from rich import print as pprint
print(pdf_result['result'].keys())  # What is this thing?
print(pdf_result['result']['contents'][0].keys()) # What's in the resulting "contents"?
print(len(pdf_result['result']['contents'][0]["annotations"])) # How many annotations are there?
pprint(pdf_result['result']['contents'][0]["annotations"][:10]) # How many annotations are there?

# with open("06-pdf-results.txt", "w") as f:
#     f.write(pdf_result['result']['contents'])
#     print('content written...')

## Human-Friendly Results - PDF Annotations

Extract and display the highlighted content and annotations from the PDF.

In [ ]:
console = Console()

if pdf_result.get("status", "").lower() == "succeeded":
    result = pdf_result.get("result", {})
    contents = result.get("contents", [])
    
    # Extract annotations from contents
    annotations = []
    for content in contents:
        for annotation in content.get("annotations", []):
            annotations.append({
                "id": annotation.get("id", "Unknown"),
                "kind": annotation.get("kind", "Unknown"),
                "author": annotation.get("author", ""),
                "createdAt": annotation.get("createdAt", "")
            })
    
    # Display annotations in a table
    if annotations:
        table = Table(title="PDF Annotations & Highlights")
        table.add_column("ID", style="cyan")
        table.add_column("Type", style="yellow")
        table.add_column("Author", style="green")
        table.add_column("Created At", style="white")
        
        for ann in annotations:
            table.add_row(
                ann["id"],
                ann["kind"],
                ann["author"],
                ann["createdAt"]
            )
        
        console.print(table)
        console.print(f"\n[bold green]Total Annotations Found: {len(annotations)}[/bold green]")
    else:
        console.print("[yellow]No annotations found in the document[/yellow]")
    
    # Display document statistics
    stats_table = Table(title="Document Statistics")
    stats_table.add_column("Metric", style="cyan")
    stats_table.add_column("Value", style="green")
    
    stats_table.add_row("Total Contents", str(len(contents)))
    stats_table.add_row("Total Annotations", str(len(annotations)))
    
    console.print("\n")
    console.print(stats_table)
else:
    console.print(f"[red]Analysis status: {pdf_result.get('status', 'Unknown')}[/red]")

# Capability 3: Image Analysis

Analyze images using the `prebuilt-imageSearch` analyzer & a gpt model.

## Configure Model Defaults

Before using image analyzers like `prebuilt-imageSearch`, we need to configure the default model deployments.
This maps the model names used by the analyzers to your actual Azure AI Foundry deployment names.

**Required models:**
- `gpt-4.1` - For most prebuilt analyzers
- `gpt-4.1-mini` - For RAG analyzers (documentSearch, imageSearch, audioSearch, videoSearch)
- `text-embedding-3-large` - For embedding operations

> **Note:** Update the deployment names in the cell below to match your actual Azure AI Foundry deployments.

In [ ]:
# Configure model deployment defaults for Content Understanding
# This is required for prebuilt analyzers like prebuilt-imageSearch
defaults_url = f"{endpoint}contentunderstanding/defaults?api-version=2025-11-01"

# First, check current defaults
response = requests.get(defaults_url, headers=headers, timeout=30)
print("Current defaults:")
print(json.dumps(response.json(), indent=2))

# Configure the model deployments
# You need to update these deployment names to match your actual Azure AI Foundry deployments
model_config = {
    "modelDeployments": {
        "gpt-4.1": "gpt-4.1",  # Update with your GPT-4.1 deployment name
        "gpt-4.1-mini": "gpt-4.1-mini",  # Update with your GPT-4.1-mini deployment name
        "text-embedding-3-large": "text-embedding-3-large"  # Update with your embedding deployment name
    }
}

patch_headers = headers.copy()
patch_headers["Content-Type"] = "application/merge-patch+json"

response = requests.patch(defaults_url, headers=patch_headers, json=model_config, timeout=30)

if response.ok:
    print("\n✓ Defaults configured successfully!")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"\n✗ Failed to configure defaults: {response.status_code}")
    print(response.text)

## Analyze Image 1: Produce

Analyze our "box of produce" image.

In [ ]:
# Image of produce
produce_image_url = "https://images.unsplash.com/photo-1635341083777-5f93a755e916?q=80&w=500"

# Download the image
image_response = requests.get(produce_image_url, timeout=30)
image_response.raise_for_status()
produce_image_data = image_response.content

print(f"Downloaded produce image: {len(produce_image_data)} bytes")

In [ ]:
# Analyze the produce image
# Note: Using prebuilt-imageSearch which provides image descriptions
image_analyze_url = f"{endpoint}contentunderstanding/analyzers/prebuilt-imageSearch:analyzeBinary?api-version=2025-11-01"

image_headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/octet-stream",
}

response = requests.post(image_analyze_url, headers=image_headers, data=produce_image_data, timeout=30)

# Check for errors and print detailed response
if not response.ok:
    print(f"Error {response.status_code}: {response.reason}")
    print(f"Response body: {response.text}")
    response.raise_for_status()

# Get the operation location
operation_location = response.headers.get("Operation-Location")

# Poll for results with progress bar
max_attempts = 30
produce_result = None
final_status = None

for attempt in tqdm(range(max_attempts), desc="Polling for results", unit="attempt"):
    time.sleep(2)
    
    result_response = requests.get(operation_location, headers=headers, timeout=30)
    result_response.raise_for_status()
    produce_result = result_response.json()
    
    final_status = produce_result.get("status", "").lower()
    
    if final_status in ["succeeded", "failed"]:
        break

# Print final status after progress bar completes
if final_status == "succeeded":
    print("\n✓ Analysis completed successfully!")
elif final_status == "failed":
    print("\n✗ Analysis failed!")
else:
    print("\n⚠ Timeout waiting for results")

## Raw API Results - Produce Image

In [ ]:
print(json.dumps(produce_result, indent=2))

## Human-Friendly Results - Produce Image

In [ ]:
from IPython.display import display, Image as IPImage

console = Console()

if produce_result.get("status") == "Succeeded":
    result = produce_result.get("result", {})
    contents = result.get("contents", [])
    
    console.print(Panel("[bold cyan]Produce Image Analysis[/bold cyan]", expand=False))
    
    # Display the analyzed image
    console.print("\n[bold green]Analyzed Image:[/bold green]")
    display(IPImage(data=produce_image_data, width=400))
    
    # Extract and display the summary from fields
    if contents:
        content = contents[0]
        fields = content.get("fields", {})
        
        # Display Summary
        summary_field = fields.get("Summary", {})
        summary_text = summary_field.get("valueString", "No summary available")
        
        console.print("\n[bold green]AI-Generated Summary:[/bold green]")
        console.print(Panel(summary_text, border_style="green"))
        
        # Display metadata
        stats_table = Table(title="Image Analysis Details")
        stats_table.add_column("Property", style="cyan")
        stats_table.add_column("Value", style="green")
        
        stats_table.add_row("Analyzer", content.get("analyzerId", "N/A"))
        stats_table.add_row("MIME Type", content.get("mimeType", "N/A"))
        stats_table.add_row("Content Kind", content.get("kind", "N/A"))
        
        console.print("\n")
        console.print(stats_table)
    else:
        console.print("[yellow]No content items found in the result[/yellow]")
else:
    console.print(f"[red]Analysis status: {produce_result.get('status', 'Unknown')}[/red]")

## Analyze Image 2: Elephants

Analyze an image containing elephants.

In [ ]:
# Image of elephants
elephants_image_url = "https://images.unsplash.com/photo-1505148230895-d9a785a555fa?q=80&w=500"

# Download the image
image_response = requests.get(elephants_image_url, timeout=30)
image_response.raise_for_status()
elephants_image_data = image_response.content

print(f"Downloaded elephants image: {len(elephants_image_data)} bytes")

In [ ]:
# Analyze the elephants image
image_analyze_url = f"{endpoint}contentunderstanding/analyzers/prebuilt-imageSearch:analyzeBinary?api-version=2025-11-01"
response = requests.post(image_analyze_url, headers=image_headers, data=elephants_image_data, timeout=30)
response.raise_for_status()

# Get the operation location
operation_location = response.headers.get("Operation-Location")

# Poll for results with progress bar
max_attempts = 30
elephants_result = None
final_status = None

for attempt in tqdm(range(max_attempts), desc="Polling for results", unit="attempt"):
    time.sleep(2)
    
    result_response = requests.get(operation_location, headers=headers, timeout=30)
    result_response.raise_for_status()
    elephants_result = result_response.json()
    
    final_status = elephants_result.get("status", "").lower()
    
    if final_status in ["succeeded", "failed"]:
        break

# Print final status after progress bar completes
if final_status == "succeeded":
    print("\n✓ Analysis completed successfully!")
elif final_status == "failed":
    print("\n✗ Analysis failed!")
else:
    print("\n⚠ Timeout waiting for results")
print(f"Operation Location: {operation_location}")

## Raw API Results - Elephants Image

In [ ]:
print(json.dumps(elephants_result, indent=2))

## Human-Friendly Results - Elephants Image

In [ ]:
console = Console()

if elephants_result.get("status", "").lower() == "succeeded":
    result = elephants_result.get("result", {})
    contents = result.get("contents", [])
    
    console.print(Panel("[bold cyan]Elephants Image Analysis[/bold cyan]", expand=False))
    
    # Display the analyzed image
    console.print("\n[bold green]Analyzed Image:[/bold green]")
    display(IPImage(data=elephants_image_data, width=400))
    
    # Extract and display the summary from fields
    if contents:
        content = contents[0]
        fields = content.get("fields", {})
        
        # Display Summary
        summary_field = fields.get("Summary", {})
        summary_text = summary_field.get("valueString", "No summary available")
        
        console.print("\n[bold green]AI-Generated Summary:[/bold green]")
        console.print(Panel(summary_text, border_style="green"))
        
        # Display metadata
        stats_table = Table(title="Image Analysis Details")
        stats_table.add_column("Property", style="cyan")
        stats_table.add_column("Value", style="green")
        
        stats_table.add_row("Analyzer", content.get("analyzerId", "N/A"))
        stats_table.add_row("MIME Type", content.get("mimeType", "N/A"))
        stats_table.add_row("Content Kind", content.get("kind", "N/A"))
        
        console.print("\n")
        console.print(stats_table)
    else:
        console.print("[yellow]No content items found in the result[/yellow]")
else:
    console.print(f"[red]Analysis status: {elephants_result.get('status', 'Unknown')}[/red]")

# Summary

This notebook demonstrated three key capabilities of Azure AI Content Understanding:

1. **Content Analyzers Discovery** - Listed all available prebuilt analyzers
2. **Content Extraction** - Extracted annotations and highlights from a PDF document
3. **Image Classification & Segmentation** - Analyzed and classified objects in images

These capabilities can be used for various document and image analysis tasks including:
- Document layout analysis
- Annotation extraction
- Object detection and classification
- Image segmentation
- Content understanding and categorization